## Monte Carlo Simulation

- Doesnt rely on assumptions, we simulate many random future paths of UK and FR prices and average the payoff.
- Kirk is good as an analytical formula but traders will often run Monte Carlo to validate it. If Kirk and MC match closely this gives us confidence in the pricing. If they diverge, something in the assumptions or inputs is off.
- Easier to boly on real-word contraints later like capacity limits, outages and transaction costs.

Summary:
- Kirk is a textbook formula, Monte Carlo is the traders sandbox that allows us to experiment with market realities.

In [72]:
import numpy as np
import pandas as pd
from scipy.stats import norm
import sys
import os
sys.path.append(os.path.abspath("/Users/Lyndon.Odia/Desktop/lo-devx/power-spread-option-pricing-main"))
from config import raw_data_dir, processed_data_dir, API_KEY, FR_DOMAIN, START_DATE, END_DATE, FX_GBP_EUR

In [73]:
# Load merged hourly dataset 
data_path = os.path.join(processed_data_dir, "UK_FR_day_ahead_hourly_merged_spread.csv")
df = pd.read_csv(data_path, parse_dates=["datetime"])

In [74]:
# Ensure sorted and clean
df = df.sort_values("datetime").reset_index(drop=True)

In [75]:
S1 = df["UK_price_eur"].mean()  # Mean UK price in EUR/MWh
S2 = df["FR_price"].mean()      # Mean FR price in EUR/MWh

# From return-based volatilities (already annualised in your EDA)
sigma1 = 0.17 #EUR/MWh
sigma2 = 0.48 #EUR/MWh
rho = 0.106   # Pearson correlation from returns

In [76]:
K = 0 # Strike price (capacity cost) in EUR/MWh - realistically this should be > 0 factoring in the actual cost to acqure interconnector rights
T = 1/12     # 1 month to expiry

In [77]:
n_sims = 100000 # Number of random variables
np.random.seed(seed=44) # seed for random number generation

Source: [Black-Scholes numerical methods notebook](https://github.com/cantaro86/Financial-Models-Numerical-Methods/blob/master/1.1%20Black-Scholes%20numerical%20methods.ipynb)

In reality, monthly transmission rights (UK-FR, or any interconnector) are typically exercisable hourly or daily as the holder decides each period whether to flow power. This means the true product payoff is the sum of many intraday spread realisations , not a single terminal payoff.

For traceability in this case study, we model the transmission right as a European spread option expiring at the end of the tenor (e.g. 1 month)

- This simplification aligns with Kirk's closed-form formula
- It allows us to compare kirk vs Monte Carlo cleanly.
- It still captures the optional value of spreads between the two markets.


(This is a common abstraction in academic and practitioner modelling: simplify to European style for intuition and benchmarking, while acknowledging that the true optionality is path-dependent)

## Summary:
• In reality - monthly transmission rights are exercised hour by hour (or day by day). Each period, the holder decides whether to flow power. The total value is the sum of all those hourly payoffs.
• In our simplified model - Treat the product as a European spread option with a single expiry at month end. This makes it much easier to price and lets us compare directly with Kirk’s formula.

## Impact on models:
• Kirk’s Approximation only works for one expiry payoff, not hourly decisions.
• Monte Carlo can handle both: in our case we use a 1-step GBM version (just expiry), but in reality you’d need a multi-step simulation to capture daily/hourly exercise.

In [78]:
# Monte Carlo Simulation
# Generate correlated standard normals
Z1 = np.random.randn(n_sims)
Zt = np.random.randn(n_sims)
Z2 = rho * Z1 + np.sqrt(1 - rho**2) * Zt

# Simulate terminal forwards (1-step Goemetric Brownian Motion) - The simulated / realised forward at expiry of the option (time T)
F1_T = S1 * np.exp(-0.5*sigma1**2*T + sigma1*np.sqrt(T)*Z1)
F2_T = S2 * np.exp(-0.5*sigma2**2*T + sigma2*np.sqrt(T)*Z2)

# Option payoff
payoffs = np.maximum(F1_T - F2_T - K, 0.0)
C_mc = payoffs.mean()

In [79]:
print(f"Monte Carlo Spread Option Value (1M): {C_mc:.2f} EUR/MWh")

Monte Carlo Spread Option Value (1M): 6.70 EUR/MWh


## Conclusion:
Monte Carlo simulation of 100,000 paths gives a 1 month UK–FR spread option value of €6.70/MWh, in line with Kirk’s Approximation (€6.71/MWh).

The close agreement validates our volatility, correlation, and expiry inputs, and shows that both analytical and numerical approaches converge on the same fair value.
For a 1 GW interconnector over a month (720 hours), this equates to a notional value of ~€4.8 million.

## Interpretation of Kirk & Monte Carlo Results
• Raw market data: UK mean = ~€82/MWh, FR mean = ~€78/MWh
• Average spread ≈ €4/MWh

## View:
• If you always flowed in one direction, we’d expect => €4/MWh ≈ €3m/month for 1 GW.
• But this ignores volatility and the ability to choose direction each hour.

## Optionality view (Kirk & Monte Carlo):
• Kirk ≈ €6.71/MWh
• Monte Carlo ≈ €6.70/MWh
• Both methods converge, validating inputs and model.
• Scaling: 1 GW × 720 hours × €6.7/MWh ≈ €4.8m/month.

## Why value > average spread:
• Capacity is an option: you only flow when profitable, never when it’s negative.
• Volatility + low correlation make optionality more valuable than the mean spread.

## Why traders pay this:
• €4.8m/month is the fair value, a benchmark and not a guaranteed profit.
• Traders pay because capacity is scarce and strategic.
• Edge comes from views (higher volatility, diverging markets, policy shifts).
• If realised spreads are wilder than history, the true value can be well above €6.7/MWh.


=> The raw spread suggested €3m/month, but the option value is €4.8m/month because of volatility and correlation. Traders pay that because capacity is scarce, and they believe their market view lets them monetise the optionality above fair value.”